# Train Model with CPU

In [ ]:
import tensorflow as tf
from tensorflow.python.client import timeline
import pylab
import numpy as np
import os

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

tf.logging.set_verbosity(tf.logging.INFO)

In [ ]:
tf.reset_default_graph()

In [ ]:
num_samples = 100000

In [ ]:
from datetime import datetime 

version = int(datetime.now().strftime("%s"))
print(version)

## Create Session

In [ ]:
config = tf.ConfigProto(
  log_device_placement=True,
)
print(config)

sess = tf.Session(config=config)
print(sess)

## Load Model Training and Validation Data 
Note the use of `shuffle_batch()`.  This uses `RandomShuffleQueue` internally.

The `min_after_dequeue` parameter is the sample size.  A larger value requires more RAM, but improves the randomness of the shuffle.

In [ ]:
training_queue = tf.train.string_input_producer([
  "hdfs://127.0.0.1:39000/linear/training.csv",
#  "hdfs://127.0.0.1:39000/linear/validation.csv",
#    "training.csv",
])

training_reader = tf.TextLineReader()
_, training_value = training_reader.read(training_queue)

x_training, y_training = tf.decode_csv(training_value, [[0.0],[0.0]])

x_training_batch, y_training_batch = \
    tf.train.shuffle_batch([x_training, y_training], 
                           batch_size=100,
                           capacity=200,
                           min_after_dequeue=100, 
                           num_threads=20)

#with tf.Session() as sess:
training_coord = tf.train.Coordinator()

training_enqueue_threads = tf.train.start_queue_runners(sess=sess, 
                                                        coord=training_coord)

print("Training Enqueue Thread Pool Size: %d" % len(training_enqueue_threads))

In [ ]:
# TODO:
# pylab.plot(x_train, y_train, '.')

In [ ]:
# TODO:
#pylab.plot(x_test, y_test, '.')

In [ ]:
# TODO:

with tf.device("/cpu:0"):
    W = tf.get_variable(shape=[], name='weights')
    print(W)

    b = tf.get_variable(shape=[], name='bias')
    print(b)

#    x_observed_batch = tf.placeholder(shape=[None], 
#                                      dtype=tf.float32, 
#                                      name='x_observed')
#    print(x_observed_batch)
    

with tf.device("/cpu:0"):
#    y_pred_batch = W * x_observed_batch + b
    y_pred_batch = W * x_training_batch + b
    print(y_pred_batch)

    
with tf.device("/cpu:0"):
#    y_observed_batch = tf.placeholder(shape=[None], dtype=tf.float32, name='y_observed')
#    print(y_observed_batch)
    
    loss_op = tf.reduce_mean(tf.square(y_pred_batch - y_training_batch))
    optimizer_op = tf.train.GradientDescentOptimizer(0.025)
    training_op = optimizer_op.minimize(loss_op)  

    print("Loss Scalar: ", loss_op)
    print("Optimizer Op: ", optimizer_op)
    print("Training Op", training_op)

## View Model Graph in Tensorboard

Navigate to the Graph tab at this URL:

http://[ip-address]:6006

## Randomly Initialize Variables (Weights and Bias)
The goal is to learn more accurate Weights and Bias during training.

In [ ]:
with tf.device("/cpu:0"):
    init_op = tf.global_variables_initializer()
    print(init_op)

sess.run(init_op)
print("W: %f" % sess.run(W))
print("b: %f" % sess.run(b))    

## View Accuracy of Initial Random Variables (Pre-Training)
This should be relatively low because we have not trained the model.

In [18]:
#def accuracy(x_param):
#  return sess.run(loss_op, 
  #feed_dict={x_observed: x_param, y_observed: y_param})
    
loss = sess.run(loss_op)
print(loss)

0.0160929


In [19]:
%%time

#try:

loss = sess.run(loss_op)

print("Accuracy: %f" % (loss*100))

#except tf.errors.OutOfRangeError as e:
#    print("Error '%s'" % str(e))
#finally:
#    training_coord.request_stop()
#    training_coord.join(enqueue_threads)

#print(1 - sess.run(loss_op))
                   #, feed_dict={x_observed: x_training_batch,
                   #             y_observed: y_training_batch}))

Accuracy: 1.723112
CPU times: user 36 ms, sys: 4 ms, total: 40 ms
Wall time: 20.3 ms


In [29]:
training_summary_writer = tf.summary.FileWriter('/root/tensorboard/linear/cpu/%s/training' % version, 
                                                 graph=tf.get_default_graph())

#validation_summary_writer = tf.summary.FileWriter('/root/tensorboard/linear/cpu/%s/validation' % version, 
#                                                 graph=tf.get_default_graph())

# TODO
#validation_summary_writer = tf.summary.FileWriter('/root/tensorboard/linear/cpu/%s/validation' % version, 
#validation_summary_writer = tf.summary.FileWriter('/root/tensorboard/linear/cpu/%s/validation' % version, 
#                                                   graph=tf.get_default_graph())

## Setup Loss Summary Operations for Tensorboard

In [21]:
loss_summary_scalar_op = tf.summary.scalar('loss', loss_op)
loss_summary_merge_all_op = tf.summary.merge_all()

## Train the Model!

In [22]:
%%time

run_metadata = tf.RunMetadata()

max_steps = 400
for step in range(max_steps - 1):
    if (step < max_steps):
        training_summary_log, _ = sess.run([loss_summary_merge_all_op, training_op]) 
#        validation_summary_log, _ = sess.run([loss_summary_merge_all_op, loss_op])
    else:  
        training_summary_log, _ = sess.run([loss_summary_merge_all_op, training_op],
                                            options=tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE), 
                                            run_metadata=run_metadata)
        
#        validation_summary_log, _ = sess.run([loss_summary_merge_all_op, loss_op])
    
    trace = timeline.Timeline(step_stats=run_metadata.step_stats)    
    with open('cpu-timeline.json', 'w') as trace_file:
        trace_file.write(trace.generate_chrome_trace_format(show_memory=True))

    if step % 10 == 0:
        training_summary_writer.add_summary(training_summary_log, step)
        training_summary_writer.flush()
#        validation_summary_writer.add_summary(validation_summary_log, step)
#        validation_summary_writer.flush()
        if step % 100 == 0:
            print(step, sess.run([W, b]))


0 [-0.3435241, 0.53272933]
100 [-0.21680978, 0.47234303]
200 [-0.1280355, 0.42268601]
300 [-0.063488364, 0.38688746]
CPU times: user 13.1 s, sys: 2.74 s, total: 15.9 s
Wall time: 11.2 s


In [23]:
#pylab.plot(x_train, y_train, '.', label="target")
#pylab.plot(x_train, sess.run(y_pred, feed_dict={x_observed: x_train, y_observed: y_train}), ".", label="predicted")
#pylab.legend()
#pylab.ylim(0, 1.0)

## View Loss Scalar Summary in Tensorboard

Navigate to the Scalars tab at this URL:

http://[ip-address]:6006

In [24]:
loss = sess.run(loss_op)
print(loss)

0.00122416


## Save Model Graph and Variables for Deployment

In [25]:
from tensorflow.python.saved_model import utils

tensor_info_x_observed = utils.build_tensor_info(x_training_batch)
print(tensor_info_x_observed)

tensor_info_y_pred = utils.build_tensor_info(y_pred_batch)
print(tensor_info_y_pred)

name: "shuffle_batch:0"
dtype: DT_FLOAT
tensor_shape {
  dim {
    size: 100
  }
}

name: "add:0"
dtype: DT_FLOAT
tensor_shape {
  dim {
    size: 100
  }
}



In [27]:
export_path = "/root/models/linear/cpu/%s" % version

print(export_path)

/root/models/linear/cpu/1495531623


In [28]:
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import signature_def_utils
from tensorflow.python.saved_model import tag_constants

with tf.device("/cpu:0"):
  builder = saved_model_builder.SavedModelBuilder(export_path)

prediction_signature =  signature_def_utils.build_signature_def(
    inputs = {'inputs': tensor_info_x_observed}, 
    outputs = {'outputs': tensor_info_y_pred}, 
    method_name = signature_constants.PREDICT_METHOD_NAME)            

legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')

builder.add_meta_graph_and_variables(sess, 
  [tag_constants.SERVING],
  signature_def_map={"predict_linear":prediction_signature,
                      signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:prediction_signature}, 
                      legacy_init_op=legacy_init_op)

builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'/root/models/linear/cpu/1495531623/saved_model.pb'


b'/root/models/linear/cpu/1495531623/saved_model.pb'

## Save Metagraph for Optimization and Transformation
We will use this later.

In [38]:
export_metagraph_path = "/root/models/optimize_me/linear/cpu/%s" % version
os.rmdir(export_metagraph_path)

os.makedirs(export_metagraph_path)

In [39]:

saver = tf.train.Saver(max_to_keep=1, keep_checkpoint_every_n_hours=24)
saver.export_meta_graph(export_metagraph_path)


FailedPreconditionError: /root/models/optimize_me/linear/cpu/1495531623.tmpa95432b921444ba68e1fb95c4ddd0fdc

## View Saved Model on Disk

You must replace `[version]` with the version number from above ^^

In [ ]:
%%bash

ls -l /root/models/linear/cpu/[version]
ls -l /root/models/linear/cpu/metagraph/

In [ ]:
#TODO

#from tensorflow.python.framework import graph_io
#graph_io.write_graph(sess.graph, "/root/models/optimize_me/", "unoptimized_cpu.pb")

## Clean Up

In [ ]:
# TODO: 
training_coord.request_stop()
training_coord.join(enqueue_threads)

#validation_coord.request_stop()
#validation_coord.join(enqueue_threads)

sess.close()

## TODO:  View Accuracy of Trained Variables
This should be relatively high after training the model.

In [ ]:
# TODO
validation_queue = tf.train.string_input_producer([
#  "hdfs://127.0.0.1:39000/linear/training.csv",
#  "hdfs://127.0.0.1:39000/linear/validation.csv",
    "validation.csv",
])

validation_reader = tf.TextLineReader()
_, validation_value = validation_reader.read(validation_queue)

x_validation, y_validation = tf.decode_csv(validation_value, [[0.0],[0.0]])

x_validation_batch, y_validation_batch = \
    tf.train.shuffle_batch([x_validation, y_validation], 
                           batch_size=100,
                           capacity=2000,
                           min_after_dequeue=1000)

validation_coord = tf.train.Coordinator()

validation_enqueue_threads = tf.train.start_queue_runners(sess=sess,
                                                          coord=validation_coord)
print("Validation Enqueue Thread Pool Size: %d" % len(validation_enqueue_threads))